In [1]:
import requests
import datetime
import base64

In [3]:
client_id = '7b119c2c66624c96b22daa923ed53848'
client_secret = '0c02558b25d248bf90e91a66979f1b28'

In [99]:
class SpotifyClient(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def getPlaylistInfo(self, playlist_id):
        endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        
        if r.status_code not in range(200, 299):
            return False, None, []
        
        data = r.json() 
        imgs = data['images']
        
        if len(imgs)!= 0:
            image_url = imgs[0]['url']
        else:
            image_url = None
        
        ids = []
        for i in data['tracks']['items']:
            ids.append(i['track']['id'])
        
        
        return True, image_url, ids
    
    def getFeatures(self, track_ids):
        ids = ','.join(track_ids)
        
        endpoint = f'https://api.spotify.com/v1/audio-features?ids={ids}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        
        if r.status_code not in range(200, 299):
            return False, []
        
        return True, r.json()['audio_features']

In [100]:
spotify = SpotifyClient(client_id, client_secret)

data = spotify.getPlaylistInfo('5AAD6wso6ksiY8V7yu6Gr2')

print(data)

In [103]:
len(data[2])

18

In [104]:
aud = spotify.getFeatures(data[2])
aud

(True,
 [{'danceability': 0.859,
   'energy': 0.628,
   'key': 10,
   'loudness': -7.199,
   'mode': 0,
   'speechiness': 0.257,
   'acousticness': 0.0545,
   'instrumentalness': 1.29e-05,
   'liveness': 0.319,
   'valence': 0.678,
   'tempo': 92.047,
   'type': 'audio_features',
   'id': '1QQxrrABKbx5w4LIphElwb',
   'uri': 'spotify:track:1QQxrrABKbx5w4LIphElwb',
   'track_href': 'https://api.spotify.com/v1/tracks/1QQxrrABKbx5w4LIphElwb',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1QQxrrABKbx5w4LIphElwb',
   'duration_ms': 233925,
   'time_signature': 4},
  {'danceability': 0.687,
   'energy': 0.793,
   'key': 2,
   'loudness': -4.254,
   'mode': 1,
   'speechiness': 0.166,
   'acousticness': 0.0603,
   'instrumentalness': 0,
   'liveness': 0.582,
   'valence': 0.751,
   'tempo': 107.045,
   'type': 'audio_features',
   'id': '5sNESr6pQfIhL3krM8CtZn',
   'uri': 'spotify:track:5sNESr6pQfIhL3krM8CtZn',
   'track_href': 'https://api.spotify.com/v1/tracks/5sNESr6pQfIhL3k